In [ ]:
#import dva_sdhdf_combine
import imp
import os
import subprocess
import h5py
import numpy as np
from astropy.time import Time
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import datetime
import matplotlib.dates as mdates
from matplotlib.dates import HourLocator as HourLocator
from matplotlib.dates import MinuteLocator as MinuteLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy import units as u
from astropy.time import TimeDelta
from scipy import interpolate
from operator import itemgetter
from itertools import groupby

day ='06'

#### Change the directory to where the files are located" ####
dir_in = '/media/ordoga/DVA_data/survey_phase1_day'+day+'/'
dir_out = '/media/ordoga/DVA_data/survey_phase1_day'+day+'/'
##############################################################

In [ ]:
scan_id = []    # The scan id number
scan_start = []  # Start time of the scan (UTC)
scan_stop = []   # Stop time of the scan (UTC)

# Read in the data and store it in arrays:
with open(dir_in+'DVAsurvey_phase1_day0'+day+'.txt') as fp:
    for line in fp:       
        scan_id.append(int(line.split()[0]))
        scan_start.append(line.split()[1]+'T'+line.split()[2][0:12])
        scan_stop.append(line.split()[3]+'T'+line.split()[4][0:12])
        
# Print out the scan numbers with their start and stop times:
for i in range(0,len(scan_id)):
    print(f"{scan_id[i]:04}",scan_start[i],scan_stop[i])

# Convert start and stop times to Modified Julian Day (MJD).
# This is needed for plotting and for selecting out data collected
# between particular times:
scan_start_mjd = Time(scan_start, format='isot',scale='utc').mjd
scan_stop_mjd  = Time(scan_stop,  format='isot',scale='utc').mjd

In [ ]:
with open(dir_in+'dva_survey_phase1_day0'+day+'_raster1.txt') as fp:
    for line in fp:  
        raster1_start = line.split()[3]
        raster1_stop  = line.split()[4]
with open(dir_in+'dva_survey_phase1_day0'+day+'_raster2.txt') as fp:
    for line in fp:  
        raster2_start = line.split()[3]
        raster2_stop  = line.split()[4]

print(raster1_start)
print(raster1_stop)
print(raster2_start)
print(raster2_stop)

raster1_start_mjd = Time(raster1_start, format='isot',scale='utc').mjd
raster1_stop_mjd  = Time(raster1_stop,  format='isot',scale='utc').mjd
raster2_start_mjd = Time(raster2_start, format='isot',scale='utc').mjd
raster2_stop_mjd  = Time(raster2_stop,  format='isot',scale='utc').mjd

In [ ]:
%%time

t_set = []
az_set = []
dec_set = []
ra_set = []
el_set = []
noise_set = []
trim_flag = []

scan0 = f"{scan_id[0]:04}"

# Use one of the scans to get the list of frequencies:
file = h5py.File(dir_in+'dva_survey_raw_scan_'+scan0+'.h5','r')
freq = file['data']['beam_0']['band_SB0']['frequency'][::12]/1e6

# Create empty arrays for the power data:
RR_set = np.empty([0,len(freq)])
LL_set = np.empty([0,len(freq)])
reRL_set = np.empty([0,len(freq)])
imRL_set = np.empty([0,len(freq)])

# Raster scan 1:
file = h5py.File(dir_in+'dva_survey_phase1_day0'+day+'_raster1'+'.h5','r')
dataset = file['data']['beam_0']['band_SB0']['scan_0']
trim_flag = dataset['metadata']['trim_scan_flag']
w = np.where(trim_flag == 0)[0]
# Add the position and time data to the corresponding arrays:
dec_set = np.concatenate([dec_set,dataset['metadata']['declination'][w]])
ra_set = np.concatenate([ra_set,dataset['metadata']['right_ascension'][w]])
el_set = np.concatenate([el_set,dataset['metadata']['elevation'][w]])
az_set = np.concatenate([az_set,dataset['metadata']['azimuth'][w]])
t_set = np.concatenate([t_set,dataset['metadata']['utc'][w]])
noise_set = np.concatenate([noise_set,dataset['metadata']['noise_state'][w]])

# Add the spectrometer power data to the corresponding arrays:
RR_set = np.concatenate([RR_set,dataset['data'][w,0,::12]],axis=0)
LL_set = np.concatenate([LL_set,dataset['data'][w,1,::12]],axis=0)
reRL_set = np.concatenate([reRL_set,dataset['data'][w,2,::12]],axis=0)
imRL_set = np.concatenate([imRL_set,dataset['data'][w,3,::12]],axis=0)


# Loop through all the scans in the "scan_num" list:
for i in scan_id:
    print(i)
    # select the file:
    file = h5py.File(dir_in+'dva_survey_raw_scan_'+f"{i:04}"+'.h5','r')
    print(file)
    
    # access the correct location in the file structure:
    dataset = file['data']['beam_0']['band_SB0']['scan_0']
    trim_flag = dataset['metadata']['trim_scan_flag']
    w = np.where(trim_flag == 0)[0]
    
    # Add the position and time data to the corresponding arrays:
    dec_set = np.concatenate([dec_set,dataset['metadata']['declination'][w]])
    ra_set = np.concatenate([ra_set,dataset['metadata']['right_ascension'][w]])
    el_set = np.concatenate([el_set,dataset['metadata']['elevation'][w]])
    az_set = np.concatenate([az_set,dataset['metadata']['azimuth'][w]])
    t_set = np.concatenate([t_set,dataset['metadata']['utc'][w]])
    noise_set = np.concatenate([noise_set,dataset['metadata']['noise_state'][w]])
    
    # Add the spectrometer power data to the corresponding arrays:
    RR_set = np.concatenate([RR_set,dataset['data'][w,0,::12]],axis=0)
    LL_set = np.concatenate([LL_set,dataset['data'][w,1,::12]],axis=0)
    reRL_set = np.concatenate([reRL_set,dataset['data'][w,2,::12]],axis=0)
    imRL_set = np.concatenate([imRL_set,dataset['data'][w,3,::12]],axis=0)
    
    
# Raster scan 2:
file = h5py.File(dir_in+'dva_survey_phase1_day0'+day+'_raster2'+'.h5','r')
dataset = file['data']['beam_0']['band_SB0']['scan_0']
trim_flag = dataset['metadata']['trim_scan_flag']
w = np.where(trim_flag == 0)[0]
# Add the position and time data to the corresponding arrays:
dec_set = np.concatenate([dec_set,dataset['metadata']['declination'][w]])
ra_set = np.concatenate([ra_set,dataset['metadata']['right_ascension'][w]])
el_set = np.concatenate([el_set,dataset['metadata']['elevation'][w]])
az_set = np.concatenate([az_set,dataset['metadata']['azimuth'][w]])
t_set = np.concatenate([t_set,dataset['metadata']['utc'][w]])
noise_set = np.concatenate([noise_set,dataset['metadata']['noise_state'][w]])

# Add the spectrometer power data to the corresponding arrays:
RR_set = np.concatenate([RR_set,dataset['data'][w,0,::12]],axis=0)
LL_set = np.concatenate([LL_set,dataset['data'][w,1,::12]],axis=0)
reRL_set = np.concatenate([reRL_set,dataset['data'][w,2,::12]],axis=0)
imRL_set = np.concatenate([imRL_set,dataset['data'][w,3,::12]],axis=0)
    
t_set_plt = Time(t_set, format='isot',scale='utc').mjd

In [ ]:
fig1, axs = plt.subplots(3,1,figsize=(20,12))

axs[0].scatter(t_set_plt,noise_set,s=5)
axs[0].set_xlim(raster1_start_mjd,raster1_stop_mjd)
axs2 = axs[0].twinx()
axs2.plot(t_set_plt,dec_set,color='k')
#axs2.set_xlim(raster1_start_mjd,raster1_stop_mjd)
axs2.set_ylim(7.5,17)

axs[1].scatter(t_set_plt,noise_set,s=5)
axs[1].set_xlim(scan_start_mjd[0],scan_stop_mjd[-1])
axs2 = axs[1].twinx()
axs2.scatter(t_set_plt,az_set,color='k',s=0.1)
#axs2.set_xlim(scan_start_mjd[0],scan_stop_mjd[-1])
axs2.set_ylim(-20,380)

axs[2].scatter(t_set_plt,noise_set,s=5)
axs[2].set_xlim(raster2_start_mjd,raster2_stop_mjd)
axs2 = axs[2].twinx()
axs2.plot(t_set_plt,dec_set,color='k')
#axs2.set_xlim(raster1_start_mjd,raster1_stop_mjd)
axs2.set_ylim(36,46)

for i in range(0,3):
    axs[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    axs[i].fmt_xdata = mdates.DateFormatter('%H:%M:%S')


In [ ]:
#print(LL_set.shape)
#print(freq.shape)
#print(t_set_plt.shape)
#print(noise_set.shape)
n_off = 5

LL_noise = np.zeros_like(LL_set)
LL_off = np.zeros_like(LL_set)
t_noise = np.zeros_like(t_set_plt)

wnoise = np.where(noise_set == 1)[0]
for k,g in groupby(enumerate(wnoise),lambda x:x[0]-x[1]):

    group = np.array(list(map(itemgetter(1),g)))
    middle = [group[int(np.floor((len(group)-1)/2))] ,group[int(np.ceil((len(group)-1)/2))]]    
    offleft = [group[0]-n_off,group[0]-1]
    offright = [group[-1]+1,group[-1]+n_off]
    
    LL_noise[middle,:] = np.nanmean(LL_set[middle,:],axis=0)
    LL_off[middle,:] = (np.nanmean(LL_set[offleft[0]:offleft[-1]+1,:],axis=0) + 
                        np.nanmean(LL_set[offright[0]:offright[-1]+1,:],axis=0))/2.
    t_noise[middle] = t_set_plt[middle]

LL_dnoise = LL_noise - LL_off
wnoise_pt = np.where(t_noise > 0)[0]
GL = LL_dnoise/np.nanmean(LL_dnoise[wnoise_pt,:],axis=0)

print(LL_dnoise.shape)
print(GL.shape)

print(t_noise)
print(t_noise[wnoise_pt].shape)
print(GL[wnoise_pt,:].shape)

GL_int = np.empty_like(GL)
for i in range(0,len(freq)):
    fint = interpolate.interp1d(t_noise[wnoise_pt],GL[wnoise_pt,i] , kind = 'cubic',fill_value="extrapolate")
    GL_int[:,i] = fint(t_set_plt)

In [ ]:
fplot = 800
df = freq[1]-freq[0]
wf = np.where(abs(freq-fplot)<df/2)[0][0]
print(freq[wf])
sz=1
fig1, axs = plt.subplots(3,1,figsize=(20,12))

axs[0].scatter(t_set_plt,LL_set[:,wf],s=sz)
axs[0].scatter(t_set_plt,LL_set[:,wf]/GL_int[:,wf],s=sz,color='red')
axs[0].scatter(t_set_plt,LL_noise[:,wf],s=sz)
axs[0].scatter(t_set_plt,LL_off[:,wf],s=sz,color='k')
ax2 = axs[0].twinx()
ax2.scatter(t_set_plt,GL[:,wf],s=5,color='k')
ax2.plot(t_set_plt,GL_int[:,wf],color='k')
axs[0].set_xlim(raster1_start_mjd,raster1_stop_mjd)
axs[0].set_ylim(0.5e7,2e7)

axs[1].scatter(t_set_plt,LL_set[:,wf],s=sz)
axs[1].scatter(t_set_plt,LL_set[:,wf]/GL_int[:,wf],s=sz,color='red')
axs[1].scatter(t_set_plt,LL_noise[:,wf],s=sz)
axs[1].scatter(t_set_plt,LL_off[:,wf],s=sz,color='k')
ax2 = axs[1].twinx()
ax2.scatter(t_set_plt,GL[:,wf],s=5,color='k')
ax2.plot(t_set_plt,GL_int[:,wf],color='k')
axs[1].set_xlim(scan_start_mjd[0],scan_stop_mjd[-1])
axs[1].set_ylim(0.5e7,3e7)

axs[2].scatter(t_set_plt,LL_set[:,wf],s=sz)
axs[2].scatter(t_set_plt,LL_set[:,wf]/GL_int[:,wf],s=sz,color='red')
axs[2].scatter(t_set_plt,LL_noise[:,wf],s=sz)
axs[2].scatter(t_set_plt,LL_off[:,wf],s=sz,color='k')
ax2 = axs[2].twinx()
ax2.scatter(t_set_plt,GL[:,wf],s=5,color='k')
ax2.plot(t_set_plt,GL_int[:,wf],color='k')
axs[2].set_xlim(raster2_start_mjd,raster2_stop_mjd)
axs[2].set_ylim(0.5e7,3.5e7)

for i in range(0,3):
    axs[i].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    axs[i].fmt_xdata = mdates.DateFormatter('%H:%M:%S')
    axs[i].grid()

In [ ]:
pnoiseLL[i,:] = np.nanmean(LL_set[wnoise_sub,:],axis=0)
pnoiseRR[i,:] = np.nanmean(RR_set[wnoise_sub,:],axis=0)
#poffLL[i,:] = np.nanmean(LL_set[list(woff1)+list(woff2),:],axis=0)
#poffRR[i,:] = np.nanmean(RR_set[list(woff1)+list(woff2),:],axis=0)
tnoise[i] = np.nanmean(t_plt[wnoise_sub])